# Taller de Física Computacional

# Sesión 2: Empezando a programar

## Funciones definidas por intervalo

Ahora vamos a integrar un poco todo eso que vimos en la Sesión 1. Vamos a escribir algunas funciones matemáticas. Epecemos por la función valor absoluto:
$$
|x| =
\left\{
	\begin{array}{ll}
		x  & \mbox{if } x \geq 0 \\
		-x & \mbox{if } x < 0
	\end{array}
\right.
$$

In [ ]:
def abs_val(x):
    if x >= 0.0:
        return x
    else:
        return -x

In [ ]:
abs_val(-5.0)

In [ ]:
abs_val(5.0)

In [ ]:
abs_val(0.0)

## La factorial

Construyamos una función que nos devuelva la factorial de un número
$$ n! = 1 \times 2 \times 3 \times\cdots\times n$$

In [ ]:
def factorial(n):
    fact = 1
    for i in range(2,n+1):
        fact = fact * i
    return fact

In [ ]:
factorial(2)

In [ ]:
factorial(45)

In [ ]:
factorial(0) # funciona!!! porqué?

In [ ]:
factorial(3.5)

Acá va una versión recursiva:

In [ ]:
def factorial_rec(n):
    if n >= 1 :
        return n*factorial_rec(n-1)
    else:
        return 1

In [ ]:
factorial_rec(5)

In [ ]:
factorial(45)

In [ ]:
factorial_rec(3.5) # OJO

In [ ]:
factorial(-5) # OJO

Comparemos con la implementación de factorial de python en el paquete `math`:

In [ ]:
import math

In [ ]:
math.factorial(5)

In [ ]:
math.factorial(45)

In [ ]:
math.factorial(3.5)

In [ ]:
math.factorial(-5)

- ¿Qué diferencias puedes encontrar entre nuestras implementaciones y las del paquete `math`?
- ¿En qué podría mejorarse nuestra implementación?
- ¿Podrías proponer una implementación mejorada?

## Evaluando una serie 

Ahora usemos nuestra factorial como bloque para construir una función que evalúe de la expansión de Taylor a orden $n$ de la función exponencial:
$$
e^x = 1 + \frac{x}{1!} + \frac{x^2}{2!} + \frac{x^3}{3!} + \cdots + \frac{x^n}{n!} = \sum_{i=0}^n \frac{x^i}{i!}
$$

In [ ]:
def exp_approx(x,n):
    resultado = 0
    for i in range(n+1):
        resultado += x**i / factorial(i)
    return resultado

Para comparar podemos usar la función exponencial que incluye python

In [ ]:
exp(3.0)

pero como vemos no está definida en en lenguaje, debemos cargar un **paquete** que contiene todas las funciones trascendentales

In [ ]:
math.exp(3.0)

In [ ]:
exp_approx(3.0,6)

In [ ]:
exp_approx(3.0,35)

podemos ser un poco más ambiciosos y pedir que la función nos devuelva una **función** que sea la aproximación de orden $n$

In [ ]:
def exp_approx(n):
    def approx(x):
        resultado = 0
        for i in range(n+1):
            resultado += x**i / factorial(i)
        return resultado
    return approx

In [ ]:
app20 = exp_approx(20)

In [ ]:
app20(3.0)

Te sugerimos desarrollar implementaciones que evalúen las series de Taylor hasta roden $n$ para las funciones $\sin$ y $\cos$.

## Evaluando una integral definida

Así como podemos devolver funciones como resultado podemos consumir funciones como argumento. Armemos una función que me devuelva el valor de la integral definida de una función arbitraria $f$ en el intervalo cerrado $[a,b]$. Vamos a utilizar una aproximación numérica de la integral definida utilizando una suma finita como aproximación. Teniendo en cuenta que
$$
\int_a^b f(\tau)d\tau = \lim_{n \rightarrow \infty} \sum_i= f(c_i) \Delta x
$$
con una partición homogénea del intervalo $[a,b]$ tal que
$$
c_i = a + \frac{(b-a)}{n} i
$$
y
$$
\Delta x = \frac{(b-a)}{n}
$$
Una función que calcula esta integral para una función arbitraria es la que sigue:

In [ ]:
def integral_definida(f,a,b,n):
    resultado = 0.0
    delta_x = (b-a)/n
    for i in range(n+1):
        c_i = a + i * (b-a)/n
        resultado += f(c_i) * delta_x
    return resultado

probemos calculando una fácil
$$
\int_0^1 exp(x) = e - 1
$$

In [ ]:
math.e - 1

In [ ]:
integral_definida(math.exp,0,1,20000)

Claramente la aproximación a la integral no es muy buena, necesitamos muchísimos intervalos para que funcione, lo que es peor es que el error depende de donde la evaluemos, por ejemplo

In [ ]:
integral_definida(math.exp,0,1,20000) - (math.e - 1)

In [ ]:
integral_definida(math.exp,0,4,20000) - (math.exp(4) - 1)

Queremos mejorar esto. Podemos hacerlo agregando un control de convergencia, incrementando el número de particiones hasta que una cierta tolerancia sea satizfecha de la siguiente manera:

In [ ]:
def integral_definida2(f,a,b,tol):
    
    n = 100 # el valor inicial de particiones
    resultado = 0.0
    delta_x = (b-a)/n
    for i in range(n+1):
        c_i = a + i * (b-a)/n
        resultado += f(c_i) * delta_x
    
    error = tol + 1.0
     
    while error > tol:
        
        n += 100
        resultado_ant = resultado
        
        resultado = 0.0
        delta_x = (b-a)/n
        for i in range(n+1):
            c_i = a + i * (b-a)/n
            resultado += f(c_i) * delta_x
        
        error = abs(resultado_ant-resultado)
        
    print("Hice",n,"particiones con un error de",error)
    return resultado

In [ ]:
def integral_definida2(f,a,b,tol):
    
    n = 0 # inicializamos n
    error = 10 * tol # nos aseguramos de que corra al menos una iteración
    resultado = 0.0 # inicializamos el resultado ¿Puece fallar esta inicialización?
     
    while error > tol:
        
        n += 100 # vamos a incrementar n de 100 a 100
        resultado_ant = resultado
        
        resultado = 0.0
        delta_x = (b-a)/n
        for i in range(n+1):
            c_i = a + i * (b-a)/n
            resultado += f(c_i) * delta_x
        
        error = abs(resultado_ant-resultado)
        
    print("Hice",n,"particiones con un error de",error)
    return resultado

In [ ]:
integral_definida2(math.exp,0,1,0.0000001) - (math.e - 1)

In [ ]:
integral_definida2(math.exp,0,1,0.00001)  - (math.e - 1)

## Para explorar:

- ¿Porqué si bien la integral converge, no converge al valor correcto?
- ¿Cómo podríamos mejorar la estimación?
- Implementa una nueva rutina que dé una mejor estimación 

## Usando Scipy

Una de las grandes ventajas del lenguaje Python es que existen una enorme variedad de librerías de funciones que resuelven una gran cantidad de problemas. En Python en general es bueno buscar y ver si el problema que estamos tratando de resolver no ha sido ya resuelto por alguien. No *reinventar la rueda*, en la jerga. Y aún en caso de que querramos reimplementar una solución por nuestra cuenta lo que existe puede servirnos para no cometer errores que otros ya han resuelto. 

El paquete [Scipy](https://docs.scipy.org/doc/scipy/reference/index.html) es tan grande que conviene pegar una mirada a la documentación para descubrir sus posibilidades. Claramente la integración numérica está implementada. Utilicemos ahora esa implementación para calcular la integral definida:

In [ ]:
from scipy import integrate

In [ ]:
integrate.quad(math.exp,0,1)[0]  - (math.e - 1)

In [ ]:
integrate.quad(math.exp,0,1)

El llamado nos devuelve una tupla con el valor de la integral y una estimación del error. La estimación del error es superior al error real, la idea es que ese sea el caso siempre para que podamos saber cuan correcto es el resultado. Como se puede ver la rutina de Scipy es mucho más rápida y precisa que nuestra implementación *naïve*. 

## Derivada numérica

Construyamos ahora una función que nos devuelva la derivada numérica de una función arbitraria. Podemos utilizar distintos métodos, el más simple es una aproximación de dos puntos que utiliza la definición de la derivada. Teniendo en cuenta que 

$$f'(x) = \lim_{h\rightarrow 0} \frac{f(x+h)-f(x)}{h}$$

Podemos aproximar la derivada de la siguiente forma:

$$f'(x) \approx \frac{f(x+h)-f(x)}{h}$$

Probemos.

In [ ]:
def derivada(f,x,h):
    return (f(x+h)-f(x))/h

In [ ]:
derivada(math.exp,1,0.001)

In [ ]:
math.e

In [ ]:
derivada(math.exp,1,0.001) - math.e

In [ ]:
derivada(math.exp,1,0.0001) - math.e

In [ ]:
derivada(math.exp,1,0.00001) - math.e

In [ ]:
derivada(math.exp,1,0.0000000000000001) - math.e

- ¿Qué pasó ahí? 
- ¿Que estimación del *orden* del error puedes hacer en base a los resultados anteriores?
- ¿Puedes encontrar una explicación de porqué salio tan mal en el último caso?

Una mejor aproximación puede hacerse de la siguiente forma. Teniendo en cuenta que el desarrollo de Taylor de taylor de $f(h+x)$ y $f(x-h)$ son:

$$ f(x+h) = f(x) + hf'(x) + \frac{1}{2}h^2f''(x)+O(h^3)+\cdots$$

$$ f(x-h) = f(x) - hf'(x) + \frac{1}{2}h^2f''(x)-O(h^3)+\cdots$$

Restando las dos ecuaciones tenemos que

$$$f'(x) \approx \frac{f(x+h) - f(x-h)}{2h} $$

In [ ]:
def derivada2(f,x,h):
    return (f(x+h)-f(x-h))/2/h

In [ ]:
derivada2(math.exp,1,0.001) - math.e

In [ ]:
derivada2(math.exp,1,0.0001) - math.e

- ¿Cual es es orden del error en esta aproximación en relación a $h$?
- ¿Porqué esta estimación es mejor que la anterior?

In [ ]:
for i in range(1,19):
    h = 10**(-i)
    print(h,derivada(math.exp,1,h) - math.e,derivada2(math.exp,1,h) - math.e)

Podemos escribir los resultados de una forma más elegante utilizando el *método* `format` del tipo `string`:

In [ ]:
for i in range(1,19):
    h = 10**(-i)
    ed1 = derivada(math.exp,1,h) - math.e
    ed2 = derivada2(math.exp,1,h) - math.e
    print("i = {3:2d}  h = {0:5.2e}  error1 = {1:9.2e}  error2 = {2:9.2e}".format(h,ed1,ed2,i))

- ¿Puedes explicar los valores de la tabla?